In [ ]:
# Importaciones necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

# Configuración de GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU detectadas: {len(gpus)}")
    except RuntimeError as e:
        print(e)
else:
    print("Usando CPU para el entrenamiento")


In [ ]:
# Parámetros de la imagen y entrenamiento
IMG_HEIGHT = 64      # Tamaño reducido para entrenar desde cero más rápido
IMG_WIDTH = 64       # Podemos usar 128x128 o 224x224 para mejor precisión
IMG_CHANNELS = 3     # RGB
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)

BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001

# Clases de clasificación
GENDER_CLASSES = ['female', 'male']
AGE_CLASSES = ['0-2', '4-6', '8-12', '15-20', '25-32', '38-43', '48-53', '60-100']

NUM_GENDER_CLASSES = len(GENDER_CLASSES)
NUM_AGE_CLASSES = len(AGE_CLASSES)

# Rutas de datos (misma estructura que antes)
GENDER_TRAIN_DIR = "dataset/gender/train"
GENDER_VAL_DIR = "dataset/gender/validation"
AGE_TRAIN_DIR = "dataset/age/train"
AGE_VAL_DIR = "dataset/age/validation"

print(f"Configuración del modelo:")
print(f"- Tamaño de imagen: {IMG_SIZE}")
print(f"- Clases de género: {NUM_GENDER_CLASSES}")
print(f"- Clases de edad: {NUM_AGE_CLASSES}")
print(f"- Batch size: {BATCH_SIZE}")
print(f"- Épocas: {EPOCHS}")
print(f"- Learning rate: {LEARNING_RATE}")


In [ ]:
def create_conv_block(input_layer, filters, kernel_size=(3, 3), pool_size=(2, 2), 
                      dropout_rate=0.25, l2_reg=1e-4):
    """
    Crea un bloque convolucional estándar
    
    Args:
        input_layer: Capa de entrada
        filters: Número de filtros convolucionales
        kernel_size: Tamaño del kernel
        pool_size: Tamaño del pooling
        dropout_rate: Tasa de dropout
        l2_reg: Regularización L2
    
    Returns:
        Capa de salida del bloque
    """
    
    # Primera convolución
    x = layers.Conv2D(
        filters, 
        kernel_size, 
        padding='same',
        kernel_regularizer=l2(l2_reg)
    )(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Segunda convolución (más profundidad)
    x = layers.Conv2D(
        filters, 
        kernel_size, 
        padding='same',
        kernel_regularizer=l2(l2_reg)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # MaxPooling y Dropout
    x = layers.MaxPooling2D(pool_size)(x)
    x = layers.Dropout(dropout_rate)(x)
    
    return x

print("Función de bloque convolucional definida")


In [ ]:
def create_custom_cnn(input_shape, num_classes, model_name="custom_cnn"):
    """
    Crea una CNN personalizada desde cero
    
    Arquitectura:
    - Input: (64, 64, 3)
    - Conv Block 1: 32 filtros → (32, 32, 32)
    - Conv Block 2: 64 filtros → (16, 16, 64)  
    - Conv Block 3: 128 filtros → (8, 8, 128)
    - Conv Block 4: 256 filtros → (4, 4, 256)
    - Global Average Pooling
    - Dense: 512 → Dropout → Dense: 256 → Dropout → Output
    
    Args:
        input_shape: Forma de entrada (height, width, channels)
        num_classes: Número de clases de salida
        model_name: Nombre del modelo
    
    Returns:
        Modelo compilado
    """
    
    # Capa de entrada
    inputs = layers.Input(shape=input_shape, name='input_layer')
    
    # Bloque 1: 32 filtros
    x = create_conv_block(inputs, 32, dropout_rate=0.25)
    
    # Bloque 2: 64 filtros  
    x = create_conv_block(x, 64, dropout_rate=0.25)
    
    # Bloque 3: 128 filtros
    x = create_conv_block(x, 128, dropout_rate=0.3)
    
    # Bloque 4: 256 filtros
    x = create_conv_block(x, 256, dropout_rate=0.3)
    
    # Global Average Pooling en lugar de Flatten (reduce overfitting)
    x = layers.GlobalAveragePooling2D()(x)
    
    # Capas densas finales
    x = layers.Dense(512, kernel_regularizer=l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(256, kernel_regularizer=l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.5)(x)
    
    # Capa de salida
    if num_classes == 2:
        # Clasificación binaria (género)
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
    else:
        # Clasificación multiclase (edad)
        outputs = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
    
    # Crear el modelo
    model = models.Model(inputs=inputs, outputs=outputs, name=model_name)
    
    return model

print("Función de creación de CNN personalizada definida")


In [ ]:
# Definir la forma de entrada
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

# === MODELO DE GÉNERO (CLASIFICACIÓN BINARIA) ===
print("=== CREANDO MODELO DE GÉNERO ===")
gender_model = create_custom_cnn(
    input_shape=input_shape,
    num_classes=2,  # Binario: female/male
    model_name="gender_cnn"
)

# Compilar modelo de género
gender_model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',  # Para clasificación binaria
    metrics=['accuracy', 'precision', 'recall']
)

print("Resumen del modelo de género:")
gender_model.summary()

print("\\n" + "="*50)

# === MODELO DE EDAD (CLASIFICACIÓN MULTICLASE) ===
print("=== CREANDO MODELO DE EDAD ===")
age_model = create_custom_cnn(
    input_shape=input_shape,
    num_classes=NUM_AGE_CLASSES,  # 8 clases de edad
    model_name="age_cnn"
)

# Compilar modelo de edad
age_model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',  # Para clasificación multiclase
    metrics=['accuracy', 'precision', 'recall']
)

print("Resumen del modelo de edad:")
age_model.summary()


In [ ]:
# Generador de datos para entrenamiento con augmentación agresiva
train_datagen = ImageDataGenerator(
    rescale=1./255,                    # Normalización
    rotation_range=30,                 # Rotación más amplia
    width_shift_range=0.3,             # Desplazamiento horizontal
    height_shift_range=0.3,            # Desplazamiento vertical
    shear_range=0.3,                   # Transformación de corte
    zoom_range=0.3,                    # Zoom aleatorio
    horizontal_flip=True,              # Volteo horizontal
    vertical_flip=False,               # No volteo vertical (rostos)
    brightness_range=[0.8, 1.2],      # Variación de brillo
    fill_mode='nearest',               # Relleno de píxeles
    validation_split=0.2               # Separar 20% para validación
)

# Generador para validación (solo normalización)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

def create_data_generators_custom(train_dir, target_size, batch_size, class_mode='categorical'):
    """
    Crea generadores de datos optimizados para entrenamiento desde cero
    
    Args:
        train_dir: Directorio de datos de entrenamiento
        target_size: Tamaño objetivo de las imágenes
        batch_size: Tamaño del lote
        class_mode: Tipo de clasificación
    
    Returns:
        train_generator, validation_generator
    """
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        subset='training',                # Usar subset de entrenamiento
        shuffle=True,
        seed=42                          # Para reproducibilidad
    )
    
    validation_generator = val_datagen.flow_from_directory(
        train_dir,                        # Mismo directorio, diferente subset
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        subset='validation',              # Subset de validación
        shuffle=False,
        seed=42
    )
    
    return train_generator, validation_generator

print("Generadores de datos personalizados configurados")


In [ ]:
def get_callbacks_custom(model_name, patience=15):
    """
    Crea callbacks optimizados para entrenamiento desde cero
    
    Args:
        model_name: Nombre del modelo para guardar
        patience: Paciencia para early stopping
    
    Returns:
        Lista de callbacks
    """
    
    callbacks = [
        # Checkpoint del mejor modelo
        ModelCheckpoint(
            filepath=f'best_{model_name}_custom.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            verbose=1
        ),
        
        # Early stopping con más paciencia para modelos desde cero
        EarlyStopping(
            monitor='val_accuracy',
            patience=patience,
            restore_best_weights=True,
            verbose=1,
            min_delta=0.001  # Mejora mínima requerida
        ),
        
        # Reducir learning rate cuando se estanque
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,          # Reducir a 30% del valor actual
            patience=8,          # Esperar 8 épocas sin mejora
            min_lr=1e-7,         # Learning rate mínimo
            verbose=1
        ),
        
        # Callback personalizado para métricas
        keras.callbacks.CSVLogger(f'{model_name}_training_log.csv')
    ]
    
    return callbacks

# Crear callbacks para ambos modelos
gender_callbacks = get_callbacks_custom('gender_model', patience=20)
age_callbacks = get_callbacks_custom('age_model', patience=20)

print("Callbacks personalizados configurados")


In [ ]:
# ENTRENAMIENTO DEL MODELO DE GÉNERO
# (Código comentado para demostración - descomentar para ejecutar)

"""
# Crear generadores de datos para género
print("Creando generadores de datos para género...")
gender_train_gen, gender_val_gen = create_data_generators_custom(
    GENDER_TRAIN_DIR,
    IMG_SIZE,
    BATCH_SIZE,
    'binary'  # Clasificación binaria para género
)

print(f"Clases encontradas: {gender_train_gen.class_indices}")
print(f"Muestras de entrenamiento: {gender_train_gen.samples}")
print(f"Muestras de validación: {gender_val_gen.samples}")

# Entrenar el modelo
print("\\nIniciando entrenamiento del modelo de género...")
history_gender = gender_model.fit(
    gender_train_gen,
    steps_per_epoch=gender_train_gen.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=gender_val_gen,
    validation_steps=gender_val_gen.samples // BATCH_SIZE,
    callbacks=gender_callbacks,
    verbose=1
)

# Guardar el modelo final
gender_model.save('custom_gender_model.h5')
print("Modelo de género guardado como 'custom_gender_model.h5'")
"""

print("Código de entrenamiento del modelo de género preparado")


In [ ]:
# ENTRENAMIENTO DEL MODELO DE EDAD
# (Código comentado para demostración - descomentar para ejecutar)

"""
# Crear generadores de datos para edad
print("Creando generadores de datos para edad...")
age_train_gen, age_val_gen = create_data_generators_custom(
    AGE_TRAIN_DIR,
    IMG_SIZE,
    BATCH_SIZE,
    'categorical'  # Clasificación multiclase para edad
)

print(f"Clases encontradas: {age_train_gen.class_indices}")
print(f"Muestras de entrenamiento: {age_train_gen.samples}")
print(f"Muestras de validación: {age_val_gen.samples}")

# Entrenar el modelo
print("\\nIniciando entrenamiento del modelo de edad...")
history_age = age_model.fit(
    age_train_gen,
    steps_per_epoch=age_train_gen.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=age_val_gen,
    validation_steps=age_val_gen.samples // BATCH_SIZE,
    callbacks=age_callbacks,
    verbose=1
)

# Guardar el modelo final
age_model.save('custom_age_model.h5')
print("Modelo de edad guardado como 'custom_age_model.h5'")
"""

print("Código de entrenamiento del modelo de edad preparado")


In [ ]:
def plot_training_metrics(history, model_name, save_plots=True):
    """
    Visualiza métricas de entrenamiento con análisis detallado
    
    Args:
        history: Historial de entrenamiento
        model_name: Nombre del modelo
        save_plots: Si guardar las gráficas
    """
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'Análisis de Entrenamiento - {model_name}', fontsize=16, y=0.98)
    
    epochs = range(1, len(history.history['accuracy']) + 1)
    
    # 1. Precisión (Accuracy)
    axes[0, 0].plot(epochs, history.history['accuracy'], 'b-', label='Entrenamiento', linewidth=2)
    axes[0, 0].plot(epochs, history.history['val_accuracy'], 'r-', label='Validación', linewidth=2)
    axes[0, 0].set_title('Precisión del Modelo')
    axes[0, 0].set_xlabel('Época')
    axes[0, 0].set_ylabel('Precisión')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Pérdida (Loss)
    axes[0, 1].plot(epochs, history.history['loss'], 'b-', label='Entrenamiento', linewidth=2)
    axes[0, 1].plot(epochs, history.history['val_loss'], 'r-', label='Validación', linewidth=2)
    axes[0, 1].set_title('Pérdida del Modelo')
    axes[0, 1].set_xlabel('Época')
    axes[0, 1].set_ylabel('Pérdida')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Precisión (Precision)
    axes[0, 2].plot(epochs, history.history['precision'], 'b-', label='Entrenamiento', linewidth=2)
    axes[0, 2].plot(epochs, history.history['val_precision'], 'r-', label='Validación', linewidth=2)
    axes[0, 2].set_title('Precisión (Precision)')
    axes[0, 2].set_xlabel('Época')
    axes[0, 2].set_ylabel('Precisión')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Recall
    axes[1, 0].plot(epochs, history.history['recall'], 'b-', label='Entrenamiento', linewidth=2)
    axes[1, 0].plot(epochs, history.history['val_recall'], 'r-', label='Validación', linewidth=2)
    axes[1, 0].set_title('Recall')
    axes[1, 0].set_xlabel('Época')
    axes[1, 0].set_ylabel('Recall')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 5. Learning Rate (si está disponible)
    if 'lr' in history.history:
        axes[1, 1].plot(epochs, history.history['lr'], 'g-', linewidth=2)
        axes[1, 1].set_title('Tasa de Aprendizaje')
        axes[1, 1].set_xlabel('Época')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].set_yscale('log')
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].text(0.5, 0.5, 'Learning Rate\\nno disponible', 
                       ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Tasa de Aprendizaje')
    
    # 6. Análisis de overfitting
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    overfitting = [abs(t - v) for t, v in zip(train_acc, val_acc)]
    
    axes[1, 2].plot(epochs, overfitting, 'purple', linewidth=2)
    axes[1, 2].set_title('Análisis de Overfitting')
    axes[1, 2].set_xlabel('Época')
    axes[1, 2].set_ylabel('Diferencia Train-Val Accuracy')
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_plots:
        plt.savefig(f'{model_name}_training_analysis.png', dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Estadísticas finales
    print(f"\\n=== ESTADÍSTICAS FINALES - {model_name} ===")
    print(f"Mejor precisión de validación: {max(val_acc):.4f}")
    print(f"Última precisión de entrenamiento: {train_acc[-1]:.4f}")
    print(f"Diferencia final train-val: {abs(train_acc[-1] - val_acc[-1]):.4f}")
    print(f"Épocas completadas: {len(epochs)}")

print("Función de visualización avanzada definida")


In [ ]:
def evaluate_model_detailed(model, test_generator, class_names, model_name):
    """
    Evaluación completa del modelo con métricas detalladas
    
    Args:
        model: Modelo entrenado
        test_generator: Generador de datos de prueba
        class_names: Lista de nombres de clases
        model_name: Nombre del modelo
    """
    
    print(f"\\n{'='*60}")
    print(f"EVALUACIÓN DETALLADA - {model_name}")
    print(f"{'='*60}")
    
    # Resetear el generador
    test_generator.reset()
    
    # Predicciones
    print("Generando predicciones...")
    predictions = model.predict(test_generator, verbose=1)
    
    # Procesar predicciones según el tipo de modelo
    if len(class_names) == 2:  # Clasificación binaria
        predicted_classes = (predictions > 0.5).astype(int).flatten()
    else:  # Clasificación multiclase
        predicted_classes = np.argmax(predictions, axis=1)
    
    # Etiquetas verdaderas
    true_classes = test_generator.classes[:len(predicted_classes)]
    
    # Métricas básicas
    accuracy = np.mean(predicted_classes == true_classes)
    print(f"\\nPrecisión general: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    # Reporte de clasificación detallado
    print(f"\\n{'-'*40}")
    print("REPORTE DE CLASIFICACIÓN:")
    print(f"{'-'*40}")
    print(classification_report(true_classes, predicted_classes, 
                              target_names=class_names, digits=4))
    
    # Matriz de confusión
    cm = confusion_matrix(true_classes, predicted_classes)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Número de predicciones'})
    plt.title(f'Matriz de Confusión - {model_name}', fontsize=14, pad=20)
    plt.xlabel('Predicción', fontsize=12)
    plt.ylabel('Etiqueta Real', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{model_name}_confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Análisis por clase
    print(f"\\n{'-'*40}")
    print("ANÁLISIS POR CLASE:")
    print(f"{'-'*40}")
    
    for i, class_name in enumerate(class_names):
        class_mask = true_classes == i
        if np.sum(class_mask) > 0:
            class_accuracy = np.mean(predicted_classes[class_mask] == true_classes[class_mask])
            print(f"{class_name}: {class_accuracy:.4f} ({class_accuracy*100:.2f}%) - {np.sum(class_mask)} muestras")
    
    # Análisis de confianza de predicciones
    print(f"\\n{'-'*40}")
    print("ANÁLISIS DE CONFIANZA:")
    print(f"{'-'*40}")
    
    if len(class_names) == 2:  # Binario
        confidence = np.maximum(predictions.flatten(), 1 - predictions.flatten())
    else:  # Multiclase
        confidence = np.max(predictions, axis=1)
    
    print(f"Confianza promedio: {np.mean(confidence):.4f}")
    print(f"Confianza mínima: {np.min(confidence):.4f}")
    print(f"Confianza máxima: {np.max(confidence):.4f}")
    print(f"Desviación estándar: {np.std(confidence):.4f}")
    
    # Distribución de confianza
    plt.figure(figsize=(10, 6))
    plt.hist(confidence, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title(f'Distribución de Confianza de Predicciones - {model_name}')
    plt.xlabel('Confianza')
    plt.ylabel('Frecuencia')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{model_name}_confidence_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return {
        'accuracy': accuracy,
        'predictions': predictions,
        'predicted_classes': predicted_classes,
        'true_classes': true_classes,
        'confidence': confidence,
        'confusion_matrix': cm
    }

print("Función de evaluación detallada definida")


In [ ]:
"""
# FLUJO DE TRABAJO COMPLETO PARA ENTRENAR MODELOS DESDE CERO

# 1. Entrenar modelo de género
print("=== ENTRENAMIENTO DEL MODELO DE GÉNERO ===")
gender_train_gen, gender_val_gen = create_data_generators_custom(
    GENDER_TRAIN_DIR, IMG_SIZE, BATCH_SIZE, 'binary'
)

history_gender = gender_model.fit(
    gender_train_gen,
    steps_per_epoch=gender_train_gen.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=gender_val_gen,
    validation_steps=gender_val_gen.samples // BATCH_SIZE,
    callbacks=gender_callbacks,
    verbose=1
)

# 2. Visualizar resultados del entrenamiento de género
plot_training_metrics(history_gender, "Modelo de Género Custom")

# 3. Evaluar modelo de género
gender_results = evaluate_model_detailed(
    gender_model, gender_val_gen, GENDER_CLASSES, "Género Custom"
)

# 4. Guardar modelo de género
gender_model.save('custom_gender_model_final.h5')

print("\\n" + "="*80)

# 5. Entrenar modelo de edad
print("=== ENTRENAMIENTO DEL MODELO DE EDAD ===")
age_train_gen, age_val_gen = create_data_generators_custom(
    AGE_TRAIN_DIR, IMG_SIZE, BATCH_SIZE, 'categorical'
)

history_age = age_model.fit(
    age_train_gen,
    steps_per_epoch=age_train_gen.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=age_val_gen,
    validation_steps=age_val_gen.samples // BATCH_SIZE,
    callbacks=age_callbacks,
    verbose=1
)

# 6. Visualizar resultados del entrenamiento de edad
plot_training_metrics(history_age, "Modelo de Edad Custom")

# 7. Evaluar modelo de edad
age_results = evaluate_model_detailed(
    age_model, age_val_gen, AGE_CLASSES, "Edad Custom"
)

# 8. Guardar modelo de edad
age_model.save('custom_age_model_final.h5')

# 9. Comparar resultados
print("\\n" + "="*80)
print("RESUMEN FINAL:")
print(f"Precisión modelo de género: {gender_results['accuracy']:.4f}")
print(f"Precisión modelo de edad: {age_results['accuracy']:.4f}")
print("="*80)
"""

print("Flujo de trabajo completo definido (comentado para demostración)")
